In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import torch.backends.cudnn as cudnn
from datasets.utils import build_dataset
from models.utils import build_model, train_model, SingleModel


gpu = 0         # gpu id, use cpu if None
seed = 1        # random seed

model_type = 'wrn'          # model name
loss_function = 'normal'    # loss function
dataset =  'cifar10'        # training dataset

# file name to save the weights and training infomation
method_name = '_'.join([dataset, model_type, loss_function, 'standard'])
print("method_name: " + method_name)

test_bs = 200       # batch size for training and testing 
num_classes = 10    # number of classes of the training dataset
input_size = 32     # input image size for the model
input_channels = 3  # number of input image channels for the model
mean=(0.492, 0.482, 0.446) # mean value for normalization of input images
std=(0.247, 0.244, 0.262)  # standard deviation for normalization of input images
prefetch_threads = 4       # number of threads used for input image preprocessing
save_path = './snapshots/' # folder path to save the weights and training information

epochs = 10             # number of epochs to train the model 
learning_rate = 0.1     # learning rate for model training
momentum = 0.9          # momentum for model training
weight_decay = 0.0005   # weight decay for model training
logitnorm_temperature = 0.04 # temperature scale of LogitNorm for model training

method_name: cifar10_wrn_normal_standard


In [3]:
# initiate the device
if gpu is not None:
    device = torch.device('cuda:{}'.format(int(gpu)))
    torch.cuda.manual_seed(seed)
else:
    device = torch.device('cpu')

# create model
net = build_model(model_type, num_classes, device, load=False, path=save_path, filename=method_name)

# load training dataset
train_data = build_dataset(dataset, mode="train", size=input_size, channels=input_channels,
                          mean=mean, std=std)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=test_bs, shuffle=False,
                                          num_workers=prefetch_threads, pin_memory=True)

# load ID test dataset for validation
test_data = build_dataset(dataset, mode="test", size=input_size, channels=input_channels,
                          mean=mean, std=std)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=test_bs, shuffle=False,
                                          num_workers=prefetch_threads, pin_memory=True)

# build training framework
framework = SingleModel(net, gpu, seed, loss_function, learning_rate, momentum, weight_decay, 
                      logitnorm_temp=logitnorm_temperature)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [4]:
train_model(epochs, framework, train_loader, test_loader,  method_name=method_name, save_path=save_path,
                save_weights=True, to_file=True, to_string=True)

NameError: name 'framework' is not defined